In [1]:
import os
from pathlib import Path

# Move to project root if we are in 'research'
if os.getcwd().endswith("research"):
    os.chdir("..")

print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: d:\OneDrive\Desktop\disaster_victim_detection


In [2]:
import dagshub
import mlflow
from urllib.parse import urlparse
from dataclasses import dataclass
from pathlib import Path
from ultralytics import YOLO
from src.victimDetector.utils.common import read_yaml, create_directories, save_json
from src.victimDetector.constants import *

# Initialize DagsHub Connection
# This configures MLflow to track directly to your repository
dagshub.init(
    repo_owner='prakashmali6556', 
    repo_name='disaster_victim_detection', 
    mlflow=True
)

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")

[2026-01-18 16:23:19,358: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as prakashmali6556

[2026-01-18 16:23:19,377: INFO: helpers: Accessing as prakashmali6556]
[2026-01-18 16:23:20,687: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/prakashmali6556/disaster_victim_detection "HTTP/1.1 200 OK"]
[2026-01-18 16:23:22,028: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "prakashmali6556/disaster_victim_detection"

[2026-01-18 16:23:23,225: INFO: helpers: Initialized MLflow to track repo "prakashmali6556/disaster_victim_detection"]


Repository prakashmali6556/disaster_victim_detection initialized!

[2026-01-18 16:23:23,228: INFO: helpers: Repository prakashmali6556/disaster_victim_detection initialized!]
MLflow Tracking URI: https://dagshub.com/prakashmali6556/disaster_victim_detection.mlflow


In [3]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_imgsz: int
    params_batch_size: int

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # ... (Keep your get_data_ingestion_config and get_prepare_base_model_config here) ...
    # ... (Keep your get_training_config here) ...

    def get_evaluation_config(self) -> EvaluationConfig:
        # 1. Get configurations from config.yaml
        training = self.config.training
        evaluation = self.config.evaluation  # <--- Now reading the evaluation section
        
        # 2. Get params
        params = self.params.yolo_params

        # 3. Locate data.yaml (Priority: Artifacts -> Root)
        data_yaml_path = os.path.join(self.config.artifacts_root, "training", "data.yaml")
        if not os.path.exists(data_yaml_path):
             data_yaml_path = os.path.join(os.getcwd(), "data.yaml")

        # 4. Create Config Object
        eval_config = EvaluationConfig(
            path_of_model=Path(training.trained_model_path),
            training_data=Path(data_yaml_path),
            
            # ✅ UPDATED: Reads dynamically from config.yaml
            mlflow_uri=evaluation.mlflow_uri,
            
            all_params=params,
            params_imgsz=params.imgsz,
            params_batch_size=params.batch_size
        )
        
        return eval_config

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def evaluation(self):
        # 1. Load the model trained in the previous step
        print(f"Loading model from: {self.config.path_of_model}")
        self.model = YOLO(self.config.path_of_model)
        
        # 2. Run Validation using YOLOv8 API
        # This calculates mAP, Precision, and Recall on the 'val' split defined in data.yaml
        self.metrics = self.model.val(
            data=self.config.training_data,
            imgsz=self.config.params_imgsz,
            batch=self.config.params_batch_size,
            split='test'
        )
        
        # 3. Save local JSON score
        self.save_score()

    def save_score(self):
        scores = {
            "map50": self.metrics.box.map50,
            "map": self.metrics.box.map,
            "precision": self.metrics.box.mp,
            "recall": self.metrics.box.mr
        }
        save_json(path=Path("scores.json"), data=scores)
        print("Scores saved to scores.json")

    def log_into_mlflow(self):
        # Ensure we are using the DagsHub URI
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        print("Logging metrics to DagsHub MLflow...")
        
        with mlflow.start_run():
            # A. Log Hyperparameters
            mlflow.log_params(self.config.all_params)
            
            # B. Log Metrics
            mlflow.log_metrics({
                "map50": self.metrics.box.map50,
                "map": self.metrics.box.map,
                "precision": self.metrics.box.mp,
                "recall": self.metrics.box.mr
            })
            
            # C. Log the Model File
            if tracking_url_type_store != "file":
                mlflow.log_artifact(self.config.path_of_model, artifact_path="model")
            else:
                mlflow.log_artifact(self.config.path_of_model, artifact_path="model")
                
        print("✅ Logging Complete! Check your DagsHub dashboard.")

In [6]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    
    # Run evaluation
    evaluation.evaluation()
    
    # Push to DagsHub
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

[2026-01-18 16:24:02,603: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-18 16:24:02,605: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-18 16:24:02,606: INFO: common: created directory at: artifacts]
Loading model from: artifacts\training\model.pt
Ultralytics 8.3.252  Python-3.10.19 torch-2.9.1+cpu CPU (Intel Core i7-10750H 2.60GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1539.6525.9 MB/s, size: 310.3 KB)
val: Scanning D:\OneDrive\Desktop\disaster_victim_detection\artifacts\data_ingestion\C2A_Dataset\val\labels.cache... 2043 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2043/2043  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 256/256 4.1it/s 1:02<0.2ss
                   all       2043      72123      0.488       0.32      0.298      0.114
Speed: 0.3ms preprocess, 15.7ms inference